In [3]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
from keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout, GlobalMaxPooling1D, \
Concatenate, BatchNormalization, Conv1D, ReLU, Input, GRU
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics import confusion_matrix
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.layers import Embedding, Flatten, Dense
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag, ne_chunk
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from datetime import datetime

/workspaces/ml-learning/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
games = pd.read_csv('datasets/steam_recommendation/games.csv')
users = pd.read_csv('datasets/steam_recommendation/users.csv').head()
recommendations = pd.read_csv('datasets/steam_recommendation/recommendations.csv')

games, users

In [ ]:
main_df = recommendations.merge(games, on='app_id')

In [ ]:
train_dataset = main_df[['is_recommended', 'helpful', 'funny', 'hours','positive_ratio','rating','linux','mac','win','price_final','price_original','discount','app_id']]
train_dataset.columns

In [ ]:
main_df.drop(columns=['review_id','user_id','date','app_id','user'], inplace=True)

In [ ]:
arr= ['is_recommended', 'linux','mac','win']
train_dataset['rating'] = LabelEncoder().fit_transform(train_dataset['rating'])
for columns in arr:
    train_dataset[columns] = to_categorical(train_dataset[columns])

train_dataset.dropna(inplace=True)
train_dataset

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(13,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# train_dataset = train_dataset.astype('float32')
# train_dataset.dtypes
np.savez_compressed('datasets/preprocessed_data/steam_game_recommendation/train_dataset.npz', data_1=train_dataset,columns=train_dataset.columns)

In [ ]:
load = np.load("/workspaces/ml-learning/datasets/train_npz.npz",allow_pickle=True)
load.files
data = load['data_1']
columns = load['columns']
#data
df = pd.DataFrame(data,columns=columns)
df
#train_dataset

In [ ]:
real_label = train_dataset['is_recommended']
real_label

In [ ]:
model.fit(train_npz, real_label, epochs=10, validation_split=0.3)